## Boosting Model

부스팅(Boosting)이란 단순하고 약한 학습기(Weak Learner)들를 결합해서 보다 정확하고 강력한 학습기(Strong Learner)를 만드는 방식의 모델이다. 

#### 1) GradientBoosting
 개별 모델로 Decision Tree 를 사용한다. 
- depth가 깊지 않은 트리를 많이 연결해서 이전 트리의 오차를 보정해 나가는 방식으로 실행한다.
- 각 모델들은 앞의 모델이 틀린 오차를 학습하여 전체 오차가 줄어들도록 학습
- 훈련시간이 많이 걸리고, 트리기반 모델의 특성상 희소한 고차원 데이터에서는 성능이 안 좋은 단점이 있다.

**주요 파라미터**
- **Decision Tree 의 가지치기 관련 매개변수**
    - 각각의 decision tree가 복잡한 모델이 되지 않도록 한다. 
- **learning_rate**
    - 이전 decision tree의 오차를 얼마나 강하게 보정할 것인지 제어하는 값. 
    - 값이 크면 보정을 강하게 하여 복잡한 모델을 만든다. 학습데이터의 정확도는 올라가지만 과대적합이 날 수있다. 
    - 값을 작게 잡으면 보정을 약하게 하여 모델의 복잡도를 줄인다. 과대적합을 줄일 수 있지만 성능 자체가 낮아질 수있다.
    - 기본값 : 0.1
- **n_estimators**
    - decision tree의 개수 지정. 많을 수록 복잡한 모델이 된다.
- **n_iter_no_change, validation_fraction** 정한 비율만큼 올라가야 성능이 올라간다고 볼건데, 그걸 몇번 안에 하는지를 정해놓고 그렇게 안하면 종료해버렸다.
    - validation_fraction에 지정한 비율만큼 n_iter_no_change에 지정한 반복 횟수동안 검증점수가 좋아 지지 않으면 훈련을 조기 종료한다.
- **보통 max_depth를 낮춰 개별 decision tree의 복잡도를 낮춘다. 보통 5가 넘지 않게 설정한다. 그리고 n_estimators를 가용시간, 메모리 한도에 맞춰 크게 설정하고 적절한 learning_rate을 찾는다.**




#### GradientBoostingClassifier

#### 1. 데이터 준비 및 X,y 생성

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
X, y = load_breast_cancer(return_X_y=True)

#### 2. 학습/검증용 데이터 분할 (Train/Test Split)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

#### 3. 모델 학습(Fit) 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Learning Rate  변화에 따른 성능 변화
import time

max_depth = 1
n_estimators = 10_000
# lr = 0.0001  # 1e-4     #학습율 조정
lr = 0.01  # 1e-2

gbc = GradientBoostingClassifier(
    n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, random_state=0
)
s = time.time()
gbc.fit(X_train, y_train)
e = time.time()

pred_train = gbc.predict(X_train)
pred_test = gbc.predict(X_test)



#### 4. 학습 결과 평가 (Score)

In [ ]:
print(f"학습률: {lr}, n_estimators: {n_estimators}, fit 시간: {e-s}초")
print_binary_classification_metrics(y_train, pred_train, title="============Train set 평가")
print_binary_classification_metrics(y_test, pred_test, title="============Test set 평가")

### 2) XGBoost
- Gradient Boost 알고리즘을 기반으로 개선해서 분산환경에서도 실행할 수 있도록 구현 나온 모델.
- Gradient Boost의 단점인 느린수행시간을 해결하고 과적합을 제어할 수 있는 규제들을 제공하여 성능을 높임.
- 회귀와 분류 모두 지원한다.
- 캐글 경진대회에서 상위에 입상한 데이터 과학자들이 사용한 것을 알려저 유명해짐.

#### 1. 데이터 준비 및 X,y 생성

In [ ]:
import xgboost
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
X, y = load_breast_cancer(return_X_y=True)

#### 2. 학습/검증용 데이터 분할 (Train/Test Split)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

#### 3. 모델 학습 (Fit)

In [ ]:
from xgboost import XGBClassifier # , XGBRegressor
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=1, random_state=0)
xgb.fit(X_train, y_train)

#### 4. 학습 결과 평가 (Score)

In [ ]:
print_binary_classification_metrics(
    y_train, xgb.predict(X_train), xgb.predict_proba(X_train)[:, 1], "Trainset"
)

#### 5. 테스트셋 최종 평가 (Best Estimator Predict)

In [ ]:
print_binary_classification_metrics(
    y_test, xgb.predict(X_test), xgb.predict_proba(X_test)[:, 1], "Test set"
)

#### feature importance 확인

In [ ]:
pd.Series(xgb.feature_importances_).sort_values(ascending=False)